In [4]:
import spacy
import requests
!python -m spacy download es_core_news_md
 
nlp = spacy.load("es_core_news_md")

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 42.3/42.3 MB 23.4 MB/s eta 0:00:00m eta 0:00:010:00:01
✔ Download and installation successful
You can now load the package via spacy.load('es_core_news_md')


## Insertamos nuestra key del api

In [1]:
api_key = "66f690904c3f75b3e102e75993bdc5a1"

## Función que realiza la llamada al api

In [27]:
from SPARQLWrapper import SPARQLWrapper, JSON

In [46]:
def get_works(writer_name):
    
    sparql = SPARQLWrapper(
        "https://query.wikidata.org/sparql"
    )
    sparql.setReturnFormat(JSON)
    
    
    query = """
    select ?workLabel
    where {{ ?s wdt:P2799 ?id . 
       ?s wdt:P1559 ?name . 
       filter regex(?name, "{0}", "i")
       BIND(uri(concat("https://data.cervantesvirtual.com/person/", ?id)) as ?bvmcID) 
       SERVICE <http://data.cervantesvirtual.com/openrdf-sesame/repositories/data> {{
          ?bvmcID <http://rdaregistry.info/Elements/a/authorOf> ?work .
          ?work rdfs:label ?workLabel
       }}
    }} limit 10
    """.format(writer_name)
    
    #print(query)
    
    sparql.setQuery(query)
    
    try:
        ret = sparql.queryAndConvert()
        
        works = ""
        for r in ret["results"]["bindings"]:
            #print('Título: ', r['workLabel']['value'])
            works += r['workLabel']['value'] + " \n"
        return works
    except Exception as e:
        print(e)
        return None

## Definimos el patrón para solicitar el tiempo

Algunos ejemplos podrían ser:

- El tiempo en Alicante
- El tiempo en Madrid

In [47]:
weather = nlp("Ejemplos de obras de autor")

print(weather.similarity(nlp("Ejemplos de obras de Miguel de Cervantes")))
print(weather.similarity(nlp("Ejemplos de obras de Lope de Vega")))

0.9607676012764527
0.9428416385211742


## Función que comprueba el tiempo

In [48]:
def chatbot(query):
    statement = nlp(query)
    min_similarity = 0.75
 
    if weather.similarity(statement) >= min_similarity:
     
        for ent in statement.ents:
            #print(ent.label_)
            if ent.label_ == "PER": 
                writer = ent.text
         
                works = get_works(writer)
                if works is not None:
                    return "Obras del escritor " + writer +": " + works
                else:
                    return "Something went wrong."
            else:
                return "Introduce un nombre de escritor."
       
    else:
        return "Lo siento, no he entendido su pregunta. Por favor, vuelva a introducir otra pregunta."

## Iniciamos el chatbot 

In [50]:
print("Hola! Soy el chatbot de escritores.........")
statement = input("Cómo puedo ayudarte ?\n")
 
response = chatbot(statement)
print(response)

Hola! Soy el chatbot de escritores.........
Cómo puedo ayudarte ?
Ejemplos de obras de Miguel de Cervantes

    select ?workLabel
    where { ?s wdt:P2799 ?id . 
       ?s wdt:P1559 ?name . 
       filter regex(?name, "Miguel de Cervantes", "i")
       BIND(uri(concat("https://data.cervantesvirtual.com/person/", ?id)) as ?bvmcID) 
       SERVICE <http://data.cervantesvirtual.com/openrdf-sesame/repositories/data> {
          ?bvmcID <http://rdaregistry.info/Elements/a/authorOf> ?work .
          ?work rdfs:label ?workLabel
       }
    } limit 10
    
Obras del escritor Miguel de Cervantes: Varias obras inéditas de Cervantes, sacadas de Códices de la Biblioteca Colombina 
Rinconete y Cortadillo 
El Quijote para todos 
Ocho entremeses de Miguel de Cervantes Saavedra 
El capitán Becerra vino a Sevilla a enseñar lo que habían de hacer los soldados, y a esto y a la entrada del duque de Medina en Cádiz hizo Cervantes este soneto 
El ingenioso hidalgo don Quixote de la Mancha 
Entremeses sele

## Referencias

https://www.askpython.com/python/examples/chatbot-in-python-using-spacy